## Imports

In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from resnet import ResNet
from training import run_epoch
import matplotlib.pyplot as plt
from data import RamanSpectraDataset  
import numpy as np
import optuna
from optuna.trial import Trial
from augment import AugmentedWrapper
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score

c:\Users\Irem\Documents\DI504\Project\Python\venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load dataset

In [3]:
data_path = "Data/ALS baseline corrected normalized merged" # Change based on your dataset location

dataset = RamanSpectraDataset(
    data_path,
    augment=False,       
    offline_aug=False     
)
sample_x, _ = dataset[0]
input_dim = sample_x.shape[-1]

print(f"Raw dataset size (before split): {len(dataset)}")
print(f"Input dimension: {input_dim}")

Raw dataset size (before split): 4500
Input dimension: 2030


In [4]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

print(f"Training samples (only raw): {len(train_set)}")

train_set = AugmentedWrapper(train_set, num_aug=2)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

print(f"Training samples (with augentation added): {len(train_set)}")
print(f"Validation samples: {len(val_set)}")
print(f"Test samples: {len(test_set)}")

Training samples (only raw): 3150
Training samples (with augentation added): 9450
Validation samples: 675
Test samples: 675


## Hyperparameter tuning with Optuna

In [ ]:
def objective(trial: Trial):
    hidden_sizes = [
        trial.suggest_categorical("hidden_1", [32, 64, 128]),
        trial.suggest_categorical("hidden_2", [64, 128, 256]),
        trial.suggest_categorical("hidden_3", [128, 256, 512])
    ]
    learning_rate = trial.suggest_float("lr", 1e-5, 1e-2, log=True)

    model = ResNet(
        hidden_sizes=hidden_sizes,
        num_blocks=[2, 2, 2],
        input_dim=input_dim,
        n_classes=1
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Short training for tuning
    for epoch in range(1, 6):
        train_loss, *_ = run_epoch(epoch, model, train_loader, cuda=torch.cuda.is_available(),
                                   training=True, optimizer=optimizer)
        val_loss, *_ = run_epoch(epoch, model, val_loader, cuda=torch.cuda.is_available(),
                                 training=False)

    return val_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

print("Best params:", study.best_params)

[I 2025-06-20 19:51:15,749] A new study created in memory with name: no-name-79d760b0-202c-4fd0-b037-7070b090b3ba
Epoch 1 Train:  13%|█▎        | 39/296 [00:03<00:18, 13.65it/s]

## Define model

In [ ]:
best_hidden = [
    study.best_params["hidden_1"],
    study.best_params["hidden_2"],
    study.best_params["hidden_3"]
]
best_lr = study.best_params["lr"]

model = ResNet(
    hidden_sizes=best_hidden,
    num_blocks=[2, 2, 2],
    input_dim=input_dim,
    n_classes=1
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=best_lr)

## Define metrics

In [ ]:
train_losses, val_losses = [], []
train_maes, val_maes = [], []
train_rmses, val_rmses = [], []
train_r2s, val_r2s = [], []
train_loss_iters, val_loss_iters = [], []

## Training

In [ ]:
for epoch in range(1, 21):
    train_loss, train_mae, train_rmse, train_r2, train_kappa, train_conf, train_batch_losses = run_epoch(
        epoch, model, train_loader, cuda=torch.cuda.is_available(), training=True, optimizer=optimizer
    )
    val_loss, val_mae, val_rmse, val_r2, val_kappa, val_conf, val_batch_losses = run_epoch(
        epoch, model, val_loader, cuda=torch.cuda.is_available(), training=False
    )

    train_loss_iters.extend(train_batch_losses)
    val_loss_iters.extend(val_batch_losses)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_maes.append(train_mae)
    val_maes.append(val_mae)
    train_rmses.append(train_rmse)
    val_rmses.append(val_rmse)
    train_r2s.append(train_r2)
    val_r2s.append(val_r2)

    print(f"Epoch {epoch}")
    print(f"  Train: Loss={train_loss:.4f}, MAE={train_mae:.4f}, RMSE={train_rmse:.4f}, R2={train_r2:.4f}, Kappa={train_kappa:.4f}")
    print(f"  Val  : Loss={val_loss:.4f}, MAE={val_mae:.4f}, RMSE={val_rmse:.4f}, R2={val_r2:.4f}, Kappa={val_kappa:.4f}")

In [ ]:
torch.save(model.state_dict(), "cv_model.ckpt")
print("Model saved as cv_model.ckpt")

## Performance evaluations

In [ ]:
epochs = range(1, 21)

def plot_loss_graph(loss_list):
    """Plot smoothed training and validation loss curves using moving average.
    Validation loss is interpolated to match the number of training iterations.
    """
    plt.figure()
    plt.xlabel('Iterations')
    plt.ylabel('Loss')

    train_loss = loss_list[0]
    val_loss = loss_list[1]

    if len(train_loss) > len(val_loss):
        val_loss_interp = np.interp(
            np.linspace(0, len(val_loss) - 1, len(train_loss)),
            np.arange(len(val_loss)),
            val_loss
        )
    else:
        val_loss_interp = val_loss

    filter_size = max(1, len(train_loss) // 10)
    kernel = np.ones(filter_size) / filter_size
    train_smoothed = np.convolve(train_loss, kernel, mode='valid')
    val_smoothed = np.convolve(val_loss_interp, kernel, mode='valid')

    plt.plot(train_smoothed, label='Train Loss')
    plt.plot(val_smoothed, label='Validation Loss')
    plt.legend()
    plt.title("Smoothed Loss Curves")
    plt.grid(True)
    plt.savefig("loss.png")

plot_loss_graph([train_loss_iters, val_loss_iters])
plt.savefig("loss_plot_smoothed.png")

def plot_metric(train_vals, val_vals, ylabel, title, filename):
    plt.figure()
    plt.plot(epochs, train_vals, label='Train')
    plt.plot(epochs, val_vals, label='Validation')
    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(filename)

plot_metric(train_losses, val_losses, "Loss", "Loss over Epochs", "loss_plot.png")
plot_metric(train_maes, val_maes, "MAE", "Mean Absolute Error over Epochs", "mae_plot.png")
plot_metric(train_rmses, val_rmses, "RMSE", "Root Mean Squared Error over Epochs", "rmse_plot.png")
plot_metric(train_r2s, val_r2s, "R²", "R² Score over Epochs", "r2_plot.png")

In [ ]:
test_loss, test_mae, test_rmse, test_r2, test_kappa, test_conf, test_batch_losses = run_epoch(
    "Test", model, test_loader, cuda=torch.cuda.is_available(), training=False
)

test_confusion_df = pd.DataFrame(test_conf, index=[-5, -6, -7, -8, -9], columns=[-5, -6, -7, -8, -9])
plt.figure(figsize=(6,5))
sns.heatmap(test_confusion_df, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Test Confusion Matrix")
plt.savefig("confusion_matrix_test.png")

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

test_preds = []
test_actuals = []

model.eval()
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x).squeeze()
        preds = [min(-5, max(-9, int(round(p.item())))) for p in outputs]
        labels = [int(round(t.item())) for t in y]

        test_preds.extend(preds)
        test_actuals.extend(labels)

print("\nClassification Report:")
target_names = ['1e-5', '1e-6', '1e-7', '1e-8', '1e-9']
print(classification_report(test_actuals, test_preds, labels=[-5, -6, -7, -8, -9], target_names=target_names))

test_acc = accuracy_score(test_actuals, test_preds)
print(f"Test Accuracy: {test_acc:.3f}")

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total trainable parameters: {count_parameters(model):,}")

## Baseline Comparison: Linear regression

In [ ]:
class SimpleLinearRegression(torch.nn.Module):
    def __init__(self, input_dim):
        super(SimpleLinearRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.linear(x)

baseline_model = SimpleLinearRegression(input_dim).to(device)
baseline_optimizer = torch.optim.Adam(baseline_model.parameters(), lr=1e-3)

baseline_train_losses, baseline_val_losses = [], []
baseline_train_batch_losses = []
baseline_val_batch_losses = []

for epoch in range(1, 21):
    train_loss, _, _, _, _, _, train_batches = run_epoch(
        epoch, baseline_model, train_loader, cuda=torch.cuda.is_available(),
        training=True, optimizer=baseline_optimizer
    )
    val_loss, _, _, _, _, _, val_batches = run_epoch(
        epoch, baseline_model, val_loader, cuda=torch.cuda.is_available(),
        training=False
    )
    baseline_train_losses.append(train_loss)
    baseline_val_losses.append(val_loss)
    baseline_train_batch_losses.extend(train_batches)
    baseline_val_batch_losses.extend(val_batches)


test_loss, test_mae, test_rmse, test_r2, test_kappa, test_conf, _ = run_epoch(
    "Test-Baseline", baseline_model, test_loader, cuda=torch.cuda.is_available(), training=False)

print("\nBaseline Linear Regression Test Results:")
print(f"  Loss={test_loss:.4f}, MAE={test_mae:.4f}, RMSE={test_rmse:.4f}, R2={test_r2:.4f}, Kappa={test_kappa:.4f}")

from sklearn.metrics import classification_report, accuracy_score

mlp_preds = []
mlp_actuals = []

baseline_model.eval()
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = baseline_model(x).squeeze()
        preds = [min(-5, max(-9, int(round(p.item())))) for p in outputs]
        labels = [int(round(t.item())) for t in y]

        mlp_preds.extend(preds)
        mlp_actuals.extend(labels)



mlp_confusion_df = pd.DataFrame(test_conf, index=[-5, -6, -7, -8, -9], columns=[-5, -6, -7, -8, -9])
plt.figure(figsize=(6,5))
sns.heatmap(mlp_confusion_df, annot=True, fmt="d", cmap="Purples")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Baseline Linear Regression Confusion Matrix")
plt.savefig("confusion_matrix_linear.png")

print("\nBaseline Linear Regression Classification Report:")
print(classification_report(mlp_actuals, mlp_preds, labels=[-5, -6, -7, -8, -9], target_names=target_names))

mlp_acc = accuracy_score(mlp_actuals, mlp_preds)
print(f"Baseline Linear Regression Test Accuracy: {mlp_acc:.3f}")

def plot_loss_graph_baseline(loss_list):
    plt.figure()
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    train_loss = loss_list[0]
    val_loss = loss_list[1]

    if len(train_loss) > len(val_loss):
        val_loss_interp = np.interp(
            np.linspace(0, len(val_loss) - 1, len(train_loss)),
            np.arange(len(val_loss)),
            val_loss
        )
    else:
        val_loss_interp = val_loss

    filter_size = max(1, len(train_loss) // 10)
    kernel = np.ones(filter_size) / filter_size
    train_smoothed = np.convolve(train_loss, kernel, mode='valid')
    val_smoothed = np.convolve(val_loss_interp, kernel, mode='valid')

    plt.plot(train_smoothed, label='Train Loss')
    plt.plot(val_smoothed, label='Validation Loss')
    plt.legend()
    plt.title("Smoothed Loss Curves (Linear Regression)")
    plt.grid(True)
    plt.savefig("loss_plot_linear.png")

plot_loss_graph_baseline([baseline_train_batch_losses, baseline_val_batch_losses])

# Ablation Study: No Augmentation

In [ ]:
raw_dataset = RamanSpectraDataset(
    data_path,
    augment=False,
    offline_aug=False
)
train_raw, val_raw, test_raw = random_split(raw_dataset, [train_size, val_size, test_size])
train_raw_loader = DataLoader(train_raw, batch_size=32, shuffle=True)
val_raw_loader = DataLoader(val_raw, batch_size=32)
test_raw_loader = DataLoader(test_raw, batch_size=32)

ablation_model = ResNet(
    hidden_sizes=best_hidden,
    num_blocks=[2, 2, 2],
    input_dim=input_dim,
    n_classes=1
).to(device)

ablation_optimizer = torch.optim.Adam(ablation_model.parameters(), lr=best_lr)

ablation_train_losses, ablation_val_losses = [], []
ablation_train_batch_losses = []
ablation_val_batch_losses = []

for epoch in range(1, 21):
    train_loss, _, _, _, _, _, train_batches = run_epoch(epoch, ablation_model, train_raw_loader, cuda=torch.cuda.is_available(), training=True, optimizer=ablation_optimizer)
    val_loss, _, _, _, _, _, val_batches = run_epoch(epoch, ablation_model, val_raw_loader, cuda=torch.cuda.is_available(), training=False)

    ablation_train_losses.append(train_loss)
    ablation_val_losses.append(val_loss)
    ablation_train_batch_losses.extend(train_batches)
    ablation_val_batch_losses.extend(val_batches)

test_loss, test_mae, test_rmse, test_r2, test_kappa, test_conf, _ = run_epoch(
    "Test-Ablation", ablation_model, test_raw_loader, cuda=torch.cuda.is_available(), training=False
)

print("\nAblation (No Augmentation) Test Results:")
print(f"  Loss={test_loss:.4f}, MAE={test_mae:.4f}, RMSE={test_rmse:.4f}, R2={test_r2:.4f}, Kappa={test_kappa:.4f}")

ablation_preds = []
ablation_actuals = []

ablation_model.eval()
with torch.no_grad():
    for x, y in test_raw_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = ablation_model(x).squeeze()
        preds = [min(-5, max(-9, int(round(p.item())))) for p in outputs]
        labels = [int(round(t.item())) for t in y]

        ablation_preds.extend(preds)
        ablation_actuals.extend(labels)


ablation_confusion_df = pd.DataFrame(test_conf, index=[-5, -6, -7, -8, -9], columns=[-5, -6, -7, -8, -9])
plt.figure(figsize=(6,5))
sns.heatmap(ablation_confusion_df, annot=True, fmt="d", cmap="Oranges")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Ablation (No Augmentation) Confusion Matrix")
plt.savefig("confusion_matrix_ablation.png")

print("\nAblation Model Classification Report:")
print(classification_report(ablation_actuals, ablation_preds, labels=[-5, -6, -7, -8, -9], target_names=target_names))

ablation_acc = accuracy_score(ablation_actuals, ablation_preds)
print(f"Ablation Test Accuracy: {ablation_acc:.3f}")

def plot_loss_graph_ablation(loss_list):
    plt.figure()
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    train_loss = loss_list[0]
    val_loss = loss_list[1]

    if len(train_loss) > len(val_loss):
        val_loss_interp = np.interp(
            np.linspace(0, len(val_loss) - 1, len(train_loss)),
            np.arange(len(val_loss)),
            val_loss
        )
    else:
        val_loss_interp = val_loss

    filter_size = max(1, len(train_loss) // 10)
    kernel = np.ones(filter_size) / filter_size
    train_smoothed = np.convolve(train_loss, kernel, mode='valid')
    val_smoothed = np.convolve(val_loss_interp, kernel, mode='valid')

    plt.plot(train_smoothed, label='Train Loss')
    plt.plot(val_smoothed, label='Validation Loss')
    plt.legend()
    plt.title("Smoothed Loss Curves (No Augmentation)")
    plt.grid(True)
    plt.savefig("loss_plot_ablation.png")

plot_loss_graph_ablation([ablation_train_batch_losses, ablation_val_batch_losses])